# Generate Training Data

We wish to speed up the process of having a human being name entities that they recognize in a paragraph, and having their positions in the paragraph identified and placed in a syntax usable by spaCy's NER training routine.

## Open PDF file, extract page two and display as sentences

In [7]:
import spacy
import PyPDF2
nlp = spacy.load('en_core_web_sm')

#Open PDF file for reading
pdfFile = open("BarCvDescLJ11.pdf", mode="rb")
pdfReader = PyPDF2.PdfFileReader(pdfFile)

# Select a page to work on
pageNumber = 8

# Get text
OnePage = pdfReader.getPage(pageNumber-1) #0-based count
OnePageText = OnePage.extractText()

# Close PDF file
pdfFile.close()

# Remove newlinesxmx82k-&a. It appears multiple newlines together makes
# Spacy think that is the end of a sentence. The PDF reader reads the text in
# an odd fashion
OnePageText = OnePageText.replace('\n','')

# create a spaCy doc object from the page and break it into sentences
doc = nlp(OnePageText)
l=0
for sent in doc.sents:
     print(l, ": ", sent)
     l = l+1


0 :  8  selected from the cross Böre/Hudson.
1 :  Its experimental designation was WA 2084-63.
2 :  It was bred to replace Hudson in the winter barley areas of eastern Washington.
3 :  It is more lodging resistant, 6 cm shorter and similar in winterhardiness compared to Hudson.
4 :  It is rough-awned, mid-tall and early maturing.
5 :  The spike is semi-dense and kernels are medium size and plump.
6 :  The aleurone is light blue and rachillas are short haired.
7 :  At the time of evaluation it was moderately resistant to scald and susceptible to stripe rust.
8 :  It was evaluated as Entry 129 in the UC Regional Cereal Testing program in 1982, 1987, and from 1989-1991 for fall planting in the intermountain region of northern California.
9 :  Crop Science 13(5):579
10 :  (1973)    KOLD  
11 :  Kold is a six-rowed winter feed barley.
12 :  It was released by the Oregon and Idaho AESs in 1993.
13 :  It was selected from the cross B-1285/Astrix.
14 :  Its experimental designation was ORWM840

## Read in Per-line named entity file and match entities to sentence positions.

In [9]:
import re
import csv
import pandas as pd
fname = "Data/barley_p8_ner.txt"

# Covert the nlp senetence generator into a list of sentences
sentences = list(doc.sents)

# Open the file of manually matched pairs (sentence # <tab> word phrase <tab> named entity)
# e.g.:
#  0      AC Metcalfe     CVAR
#  0      two-rowed       TRAT
#  0      barley          CROP
#  1      Agri-Food Candada   ORG
#  1      1997    DATE
file = open(fname)
reader = csv.reader(file, delimiter='\t', quoting=csv.QUOTE_NONE)
data = list()

for row in reader:
    try:
        (sentIndex, phrase, label) = row
        sent = sentences[int(sentIndex)].string.rstrip()
        
        # find all instances of the 'phrase' in the 'sent'.
        iter = re.finditer(r"\b"+phrase+r"\b", sent)
        indices = [m.start(0) for m in iter]
        
        # check to make sure the phrase the user said was there was indeed found
        if len(indices) == 0:
            raise ValueError
                
        # print out all instances
        for i in indices:
#            print(sentIndex, sent, phrase, "("+str(i), i+len(phrase), "'"+label+"')")
            data.append([sentIndex, sent, phrase, "("+str(i)+", "+str(i+len(phrase))+", '"+label+"')"])
            
    except:
        print("Handle manually: ", row)
        
df = pd.DataFrame(data, columns = ["Index", "Sentence", "Phrase", "MatchInfo"])
print(df)


Handle manually:  ['9', 'Crop Science 13(5):579 (1973)', 'JRNL']
Handle manually:  ['21', 'Crop Science 35(5):1503 (1995)', 'JRNL']
Handle manually:  ['44', 'Crop Science 12:124 (1972)', 'JRNL']
Handle manually:  ['64', 'Crop Science 26:1079 (1986)']
    Index                                           Sentence       Phrase  \
0       0            8  selected from the cross Böre/Hudson.  Böre/Hudson   
1       1       Its experimental designation was WA 2084-63.   WA 2084-63   
2       2  It was bred to replace Hudson in the winter ba...       Hudson   
3       2  It was bred to replace Hudson in the winter ba...       winter   
4       2  It was bred to replace Hudson in the winter ba...       barley   
..    ...                                                ...          ...   
144    62  At the time of evaluation it was moderately re...        scald   
145    62  At the time of evaluation it was moderately re...  susceptible   
146    62  At the time of evaluation it was moderately r

## Create a function to clean up overlapping intervals

In [10]:
import re
coordRegex = re.compile(r'(\d+), (\d+)')

def sortByStart(coords):
    """For use in sort routines, return object with lowest (X,Y) values"""
    # split out coordinates that come in as (5, 7, 'CVAR')
    mo = coordRegex.search(coords)
    return(int(mo.group(1)))

def overlaps(coord1, coord2):
    """Check if coordinates of the form 5, 7, 'CVAR' and 32, 46, 'TRAT' overlap"""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
    
    if ((coord1High >= coord2Low) and (coord1Low <= coord2Low) or
        (coord2High >= coord1Low) and (coord2Low <= coord1Low)):
        return True
    else:
        return False

def keepFirst(coord1, coord2):
    """Given overlapping coordinates, return the wider encompassing one."""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
 
    if (int(coord1High) - int(coord1Low)) >= (int(coord2High) - int(coord2Low)):
        return True
    else:
        return False

# print("Should be false:", overlaps("(5, 7, 'CVAR')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(26, 46, 'TRAT')"))
# print("Keeper:", keepFirst("34, 46, 'TRAT'", "34, 46, 'TRAT'"))

def cleanIntervals(inputString=""):
    """order intervals like (5, 7, 'CVAR'), (32, 46, 'TRAT'), (26, 46, 'TRAT') and remove overlapping ones."""
    inputString = inputString.lstrip("(").rstrip(")")
    intervalList = inputString.split("), (")
    intervalList.sort(key = sortByStart)
#    print("Sorted Interval List:", intervalList)

    # Pairwise compare every interval in the list to every other interval to check overlap
    keeperList = [True]*len(intervalList) # Logic array to determine if each interval should be kept
    i=0
    for interval1 in intervalList:
        for interval2 in intervalList:
            if interval1 == interval2:
                if intervalList.index(interval1) != i: # when both are the same we reject the higher one
                    keeperList[i] = False
            else:
                if overlaps(interval1, interval2) and keepFirst(interval1, interval2) == False:
                    keeperList[i] = False
        i = i+1
        
#    print("keeperList:", keeperList)
   
    # Build up the return interval list
    returnStr = "("
    for interval, isKeeper in zip(intervalList, keeperList):
        if isKeeper:
            returnStr = returnStr + interval + "), ("
    return (returnStr.rstrip("), (") + ")")
        
# cleanIntervals("(5, 7, 'CVAR'), (32, 46, 'TRAT'), (5, 9, 'CVAR'), (48, 55, 'ORG'), (26, 46, 'TRAT')")
# cleanIntervals("(0, 8, 'CVAR'), (0, 5, 'CVAR'), (21, 26, 'PLAN'), (32, 37, 'CVAR')")
cleanIntervals("(0, 12, 'CVAR'), (39, 49, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (140, 150, 'CVAR'), (181, 187, 'CVAR'), (181, 187, 'CVAR')")


"(0, 12, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (181, 187, 'CVAR')"

## Aggregate all matches for each sentence on a single line and output in spaCy training format

In [11]:
# use Pandas dataframes to aggregate all entity matches together for a single sentence
agg_rules = {'Sentence': 'first', 'Phrase': 'first', 'MatchInfo': lambda x: ', '.join(x)}
res = df.groupby('Index').agg(agg_rules)
#print(res)

# Now format it just like what is needed for the spaCy training module: 
# E.g.:
# ('Eight-Twelve is a six-rowed winter feed barley', {'entities': [(0, 12, 'CVAR'), (18, 27, 'TRAT'), (28, 39, 'TRAT'),(40, 46, 'CROP')]}),
records = res.to_dict('records')
for i in range(0,len(records)):
    print("    ('"+records[i]['Sentence']+"', {'entities': ["+cleanIntervals(records[i]['MatchInfo'])+"]}),")
    #print("    ('"+records[i]['Sentence']+"', {'entities': ["+records[i]['MatchInfo']+"]}),")


    ('8  selected from the cross Böre/Hudson.', {'entities': [(27, 38, 'PED')]}),
    ('Its experimental designation was WA 2084-63.', {'entities': [(33, 43, 'ALAS')]}),
    ('Kold is a six-rowed winter feed barley.', {'entities': [(0, 4, 'CVAR'), (10, 19, 'TRAT'), (20, 26, 'TRAT'), (27, 31, 'TRAT'), (32, 38, 'CROP')]}),
    ('It was released by the Oregon and Idaho AESs in 1993.', {'entities': [(23, 44, 'ORG'), (48, 52, 'DATE')]}),
    ('It was selected from the cross B-1285/Astrix.', {'entities': [(31, 44, 'PED')]}),
    ('Its experimental designation was ORWM8407.', {'entities': [(33, 41, 'ALAS')]}),
    ('It has midseason maturity and mid-short height.', {'entities': [(17, 25, 'TRAT'), (40, 46, 'TRAT')]}),
    ('It has moderate resistance to lodging and fair winterhardiness.', {'entities': [(16, 37, 'TRAT'), (47, 62, 'TRAT')]}),
    ('The spikes are lax with rough awns.', {'entities': [(4, 10, 'PLAN'), (30, 34, 'PLAN')]}),
    ('Aleurone is white.', {'entities': [(0, 8, 'PLAN')]}),